# Backpropogation using real-valued computational graph 

```
 dQ/dX
X ---\
      \    df/dQ
       (+)---\
 dQ/dY/   Q   \
Y ---/         \
                (*)--- f(x, y, z)
               /   df/df
 df/dZ        /
Z -----------/
```

#### By Chain rule
```
given Q = X + Y

df/dQ = df/df * df/dQ
df/dX = df/dQ * dQ/dX
df/dY = df/dQ * dQ/dY
df/dZ = df/df * df/dZ
```
by manually calculating the partial derivatives...
```
df/dQ = Z
df/dX = Z * 1
df/dY = Z * 1
df/dZ = X + Y
```

In [2]:
def forward(x, y, z):
    return z*(x + y)

In [3]:
def backward(x, y, z):
    dfdx = z
    dfdy = z
    dfdz = x + y
    return dfdx, dfdy, dfdz

In [4]:
x = 2
y = 5
z = 9
dx, dy, dz = backward(x, y, z)
print ("Gradients on X: {}, Y: {} and Z: {}".format(dx, dy, dz))

Gradients on X: 9, Y: 9 and Z: 7


---
# Converting the above computation graph to an optimization problem

### This can simply be done by adding a loss layer at the end. Lets say we add a mean square loss `node` after `f(x, y, z)`. Here we define value `V` that we want to optimize using the function `f(x, y, z)`. Below is how the new computational graph would look like. Notice that we are taking derivatives w.r.t `L` not `f` anymore.

```
 dQ/dX
X ---\
      \    df/dQ
       (+)---\
 dQ/dY/   Q   \
Y ---/         \
                (*)--- f(x, y, z)---Loss
               /   dL/df
        df/dZ /
Z -----------/
```

```
L = (V - f(x,y,z))^2
dL/dQ = dL/df * df/dQ
dL/dX = dL/dQ * dQ/dX
dL/dY = dL/dQ * dQ/dY
dL/dZ = dL/df * df/dZ
```

analytically this becomes...

```
dL/dQ = -2(V-f)*Z
dL/dX = -2(V-f)*Z*1 = dL/dQ*Z
dL/dY = -2(V-f)*Z*1 = dL/dQ*Z
dL/dZ = -2(V-f)*(X + Y)

```

now the backward calculation changes to below. You can verify this using wolframalpha

In [5]:
def backward_w_loss(V, x, y, z):
    dldx = -2 * z * (V - (x + y) * z)
    dldy = -2 * z * (V - (x + y) * z)
    dldz = -2 * (x + y) * (V - (x + y) * z)
    return dldx, dldy, dldz

# Optimization loop for variables `x`, `y` and `z`

Here I am optimizing the variables to produce a `final_value` of `144`. Which is partly equivalent to training a neural network to learn a certain output/final_value. Notice how in the iteration, you do not need the `forward` function (only in this case because it is a simple/small network and we are solving the entire network analytically). If you were do this piecewise as in a `real` neural network, you need to do the `forward` pass to calculate the loss, which will then get backpropogated through the network.

Play around with all initial values of the variables (long enough) and you will essentially find all the real values of `x`, `y` and `z` that satisfy the equation. 

Notice that this problem has the same steps as the previous curvefitting assigment. Here instead you are training a computational graph to approximate an output value. Also a more complex computational graph like a neural network will follow the same steps.

In [6]:
x = 12.3791
y = 1.4782
z = 8.192
alpha = 0.001
final_value = 144
n = 1000
for i in range(n):
    dx, dy, dz = backward_w_loss(final_value, x, y, z)
    x = x - alpha * dx
    y = y - alpha * dy
    z = z - alpha * dz
print ("Final Values of x: {}, y: {} and z: {}".format(x, y, z))
# forward is only used here below
print ("Evaluation of (x + y) * z = {}".format(forward(x, y, z)))

Final Values of x: 13.1042097033, y: 2.20330970331 and z: 9.40714142997
Evaluation of (x + y) * z = 144.0


# BackPropogation on different and much complex computational graph


```
 
X1 ---\
       \
       (*)----\
       /       \
W1 ---/         \
                 \
                 (-)-----\
                 /        \  
X2 ----\        /          \
        \      /            \
        (*)---/             (+)------ tanh(O) ----- Loss
        /                   /
W2 ----/                   /    
                          /
                         /
X3 ---------------------/


X1, X2, X3 are inputs
W1, W2 are weights 
```


## Hyperbolic Tangent non-linearity
$$\tanh(x) = \frac{e^{2x}-1}{e^{2x}+1}$$

$$\frac{\delta \tanh(x)}{\delta x} = \frac{4 e^{2x}}{(e^{2x}+1)^2}$$

In [7]:
import numpy as np

In [8]:
class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad

In [9]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value * self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = self.x2.value * self.output.gradient
        self.x2.gradient = self.x1.value * self.output.gradient

In [10]:
class AddNode(object):    
    """
    Adds two inputs x1 and x2.
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value + self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = 1 * self.output.gradient
        self.x2.gradient = 1 * self.output.gradient

In [11]:
class SubtractNode(object):    
    """
    subtract two inputs x1 and x2.
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value - self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = 1 * self.output.gradient
        self.x2.gradient = -1 * self.output.gradient

In [12]:
class HyperbolicTangentNode(object):
    """
    Adds a Hyperbolic Tangent non-linearity to a single input
    """
    def forward(self, x):
        self.x = x
        self.output = Node(((np.exp(2 * self.x.value) - 1)/ (np.exp(2 * self.x.value) + 1)), 0.0)
        return self.output
        
    def backward(self):
        t = (4 * (np.exp(2 * self.x.value)))/ ((np.exp(2 * self.x.value) + 1) ** 2)
        self.x.gradient = t * self.output.gradient

In [13]:
def forward_nn():
    # w1 * x1
    w1x1 = w1_mul_x1.forward(w1, x1)
    # w2 * x2
    w2x2 = w2_mul_x2.forward(w2, x2)
    # w1*x1 - w2*x2
    w1x1_w2x2 = w1x1_subtract_w2x2.forward(w1x1, w2x2)
    # w1*x1 - w2*x2 + x3
    w1x1_w2x2_x3 = w1x1w2x2_add_x3.forward(w1x1_w2x2, x3)
    # HyperbolicTangent(w1*x1 - w2*x2 + x3)
    output = HyperbolicTangent_out.forward(w1x1_w2x2_x3)
    return output

def backward_nn():
    HyperbolicTangent_out.backward()
    w1x1w2x2_add_x3.backward()
    w1x1_subtract_w2x2.backward()
    w2_mul_x2.backward()
    w1_mul_x1.backward()

In [14]:
# Initialize Weights and Bias
w1 = Node(0.1, 0.0)
w2 = Node(0.4, 0.0)

# Input/Target Output
alpha = 0.001
x1 = Node(1.0, 0.0)
x2 = Node(1.0, 0.0)
x3 = Node(-0.02, 0.0)
final_value = 0.475

# Create Nodes
w1_mul_x1 = MultiplyNode()
w2_mul_x2 = MultiplyNode()
w1x1_subtract_w2x2 = SubtractNode()
w1x1w2x2_add_x3 = AddNode()
HyperbolicTangent_out = HyperbolicTangentNode()

In [15]:
for i in range(1000000):
    forward_output = forward_nn()
    forward_output.gradient = -2 * (final_value - forward_output.value)
    backward_nn()
    w1.value -= alpha * w1.gradient
    w2.value -= alpha * w2.gradient
    
forward_output.value

0.47499999999999887